In [590]:

import string
from nltk import word_tokenize
from nltk.stem import PorterStemmer

trn_dict = {key: value for key, value in zip(list(string.punctuation), [None]*len(string.punctuation))}
def process_text(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    text = text.translate(trn_dict)
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens

In [592]:

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

# Transform texts to Tf-Idf coordinates
vectorizer = TfidfVectorizer(tokenizer=process_text,
                             stop_words=stopwords.words('english'),
                             max_df=0.5,
                             min_df=0.1,
                             lowercase=True)
tfidf_model = vectorizer.fit_transform(basic_quals_df.qualification_str.tolist())

C:\Users\577342\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'ha', 'hi', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'thi', 'veri', 'wa', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [655]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
])
sents_list = basic_quals_df.qualification_str.tolist()
X = pipeline.fit_transform(sents_list).toarray()
#print(['X.{}'.format(fn) for fn in dir(X) if fn.startswith('to')])
y = basic_quals_df.is_fit.to_numpy()

In [659]:


# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.
vectorizer = CountVectorizer(analyzer='word', tokenizer=None, preprocessor=None,
                             stop_words=None, max_features=5000)

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of
# strings.
qual_features = vectorizer.fit_transform(sents_list)
print(type(qual_features))

<class 'scipy.sparse.csr.csr_matrix'>


In [670]:

import scipy


idf_cut = 90 #percentile, we will only consider words with an idf >= the ___ percentile of the idfs of all words
vectorizer = TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1,2)) #unigrams and bigrams
#vectorizer = TfidfVectorizer(min_df=1, stop_words='english', analyzer='word', tokenizer=tokenize) #just unigrams (words)
X = vectorizer.fit_transform(sents_list)
print(['vectorizer.{}'.format(fn) for fn in dir(vectorizer) if 'idf' in fn.lower()])
idf = vectorizer.idf_
idf_df = pd.DataFrame(columns=['word','idf'])
idf_df.word = vectorizer.get_feature_names()
idf_df.idf = idf
idf_dict = dict(zip(vectorizer.get_feature_names(), idf))
select_terms = list(idf_df.word.loc[idf_df.idf >= scipy.stats.scoreatpercentile(idf, idf_cut)]) #let's only consider words with high idf

['vectorizer._tfidf', 'vectorizer.idf_', 'vectorizer.smooth_idf', 'vectorizer.use_idf']


In [671]:

vectorizer = CountVectorizer(min_df=1, vocabulary=select_terms, ngram_range=(1,2), stop_words='english')
#vectorizer = CountVectorizer(min_df=1, vocabulary=select_terms, stop_words='english', analyzer='word', tokenizer=tokenize)
X = vectorizer.fit_transform(sents_list)
print(['vectorizer.{}'.format(fn) for fn in dir(vectorizer) if 'idf' in fn.lower()])

[]


In [660]:

print(['qual_features.{}'.format(fn) for fn in dir(qual_features) if not fn.startswith('_')])

['qual_features.arcsin', 'qual_features.arcsinh', 'qual_features.arctan', 'qual_features.arctanh', 'qual_features.argmax', 'qual_features.argmin', 'qual_features.asformat', 'qual_features.asfptype', 'qual_features.astype', 'qual_features.ceil', 'qual_features.check_format', 'qual_features.conj', 'qual_features.conjugate', 'qual_features.copy', 'qual_features.count_nonzero', 'qual_features.data', 'qual_features.deg2rad', 'qual_features.diagonal', 'qual_features.dot', 'qual_features.dtype', 'qual_features.eliminate_zeros', 'qual_features.expm1', 'qual_features.floor', 'qual_features.format', 'qual_features.getH', 'qual_features.get_shape', 'qual_features.getcol', 'qual_features.getformat', 'qual_features.getmaxprint', 'qual_features.getnnz', 'qual_features.getrow', 'qual_features.has_canonical_format', 'qual_features.has_sorted_indices', 'qual_features.indices', 'qual_features.indptr', 'qual_features.log1p', 'qual_features.max', 'qual_features.maximum', 'qual_features.maxprint', 'qual_fe

In [657]:

pipeline.predict()

AttributeError: 'TfidfTransformer' object has no attribute 'predict'

In [629]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

clf = RandomForestClassifier(n_estimators=1000, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [630]:

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[7 2]
 [4 7]]
              precision    recall  f1-score   support

           0       0.64      0.78      0.70         9
           1       0.78      0.64      0.70        11

    accuracy                           0.70        20
   macro avg       0.71      0.71      0.70        20
weighted avg       0.71      0.70      0.70        20

0.7


In [638]:

vectorizer.get_feature_names()

[',', 'abil', 'develop', 'includ', 'knowledg', 'python', 'softwar', 'year']